<a href="https://colab.research.google.com/github/hjh1009/UTS_ML2019_ID12794881/blob/master/A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.colors
import matplotlib.pyplot as plt



data = np.loadtxt('HeightWeight.csv', dtype=np.float, delimiter=',', skiprows=1)
print (data.shape)
y, x = np.split(data, [1, ], axis=1)
x, x_test, y, y_test = train_test_split(x, y, train_size=0.6, random_state=0)
    
em = GaussianMixture(n_components=2, covariance_type='full', random_state=0)
em.fit(x)
x_min = np.min(x, axis=0)
x_max = np.max(x, axis=0)

print ('means = \n', em.means_)
print ('covariances = \n', em.covariances_)
    
y_pre = em.predict(x)
y_test_pre = em.predict(x_test)
gaibian = (em.means_[0][0] > em.means_[1][0])
if gaibian:
    z = y_pre == 0
    y_pre[z] = 1
    y_pre[~z] = 0
    z = y_test_pre == 0
    y_test_pre[z] = 1
    y_test_pre[~z] = 0
acc = np.mean(y_pre.ravel() == y.ravel())
acc_test = np.mean(y_test_pre.ravel() == y_test.ravel())
acc_str = u'train-acc：%.2f%%' % (acc * 100)
acc_test_str = u'test-acc：%.2f%%' % (acc_test * 100)
print (acc_str)
print (acc_test_str)

cm_light = mpl.colors.ListedColormap(['#FF8080', '#77E0A0'])
cm_dark = mpl.colors.ListedColormap(['r', 'g'])
N, M = 200, 200                                # How many values are sampled horizontally
x1_min, x1_max = x[:, 0].min(), x[:, 0].max()  # Range of column 0
x2_min, x2_max = x[:, 1].min(), x[:, 1].max()  # Range of column 1
t1 = np.linspace(x1_min, x1_max, N)
t2 = np.linspace(x2_min, x2_max, M)
x1, x2 = np.meshgrid(t1, t2)                   # Generate grid sampling points
grid_test = np.stack((x1.flat, x2.flat), axis=1)  # Grid point
print(x1,x2,grid_test)
grid_pre = em.predict(grid_test)
grid_pre = grid_pre.reshape(x1.shape)

if gaibian:
    z = grid_pre == 0
    grid_pre[z] = 1
    grid_pre[~z] = 0
plt.figure(figsize=(10, 8), facecolor='w')
plt.pcolormesh(x1, x2, grid_pre, cmap=cm_light)
plt.scatter(x[:, 0], x[:, 1], s=50, c=y.ravel(), marker='o', cmap=cm_dark, edgecolors='k')
plt.scatter(x_test[:, 0], x_test[:, 1], s=60, c=y_test.ravel(), marker='^', cmap=cm_dark, edgecolors='k')

# Painting contour map
p = em.predict_proba(grid_test)
np.set_printoptions(suppress=True)
print(p)
p = p[:, 0].reshape(x1.shape)
QS = plt.contour(x1, x2, p, levels=(0.2, 0.5, 0.8), colors=list('rgb'), linewidths=2)
plt.clabel(QS, fontsize=15, fmt='%.1f', inline=True)
ax1_min, ax1_max, ax2_min, ax2_max = plt.axis()
xx = 0.9*ax1_min + 0.1*ax1_max
yy = 0.1*ax2_min + 0.9*ax2_max
plt.text(xx, yy, acc_str, fontsize=18)
yy = 0.15*ax2_min + 0.85*ax2_max
plt.text(xx, yy, acc_test_str, fontsize=18)
plt.xlim((x1_min, x1_max))
plt.ylim((x2_min, x2_max))
plt.xlabel(u'Height(cm)', fontsize='large')
plt.ylabel(u'Weight(kg)', fontsize='large')
plt.title(u'EM algorithm estimates GMM parameters', fontsize=20)
plt.grid() 
plt.show()